In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import os
import plotly.express as px

In [2]:
# Setting the Data files
counties_population_path = os.path.join("Resources","co-est2019-annres.xlsx")
covid_counties_path = os.path.join("Resources","us-counties.csv")

In [3]:
# Read the population data and the case results
population_metadata = pd.read_excel(counties_population_path)
case_results = pd.read_csv(covid_counties_path)

In [4]:
population_metadata

,Geographic Area,2018,2019
0,United States,326687501,328239523
1,".Autauga County, Alabama",55533,55869
2,".Baldwin County, Alabama",217855,223234
3,".Barbour County, Alabama",24872,24686
4,".Bibb County, Alabama",22367,22394
...,...,...,...
3138,".Sweetwater County, Wyoming",42858,42343
3139,".Teton County, Wyoming",23269,23464
3140,".Uinta County, Wyoming",20292,20226
3141,".Washakie County, Wyoming",7877,7805
